In [2]:
import os
! git clone https://github.com/martin-marek/parking-space-occupancy
os.chdir('parking-space-occupancy')

fatal: destination path 'parking-space-occupancy' already exists and is not an empty directory.


In [ ]:
import torch, os, requests
from models.rcnn import RCNN
from utils import transforms, transform_json, import_json
from PIL import Image
import numpy as np
from utils import visualize as vis 
import torchvision

# create model
model = RCNN()

# load model weights
weights_path = 'weights.pt'
if not os.path.exists(weights_path):
    r = requests.get('https://storage.googleapis.com/pd-models/RCNN_128_square_gopro.pt')  
    with open(weights_path, 'wb') as f:
        f.write(r.content)
model.load_state_dict(torch.load(weights_path, map_location='cpu'))

# inference
image_path = "./dataset/data/images/ParkProva.jpg"
image = torchvision.io.read_image(image_path)

rois = transform_json.ultralytics_to_tensor('./dataset/data/bounding_boxes.json')
rois = transform_json.normalize_tensor_with_image(rois, image_path)

repo_path = "./RestPKLOTs-1/test/images"

for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                image = torchvision.io.read_image(image_path)
                rois = import_json.extract_and_standardize_bboxes('./RestPKLOTs-1/test/_annotations.coco.json', image_path)

                # Preprocess the image
                image = transforms.preprocess(image)
                image_tensor = transforms.preprocess(T.ToTensor()(image).unsqueeze(0))

                # Perform inference
                with torch.no_grad():
                    class_logits = model(image_tensor, rois)
                    class_scores = class_logits.softmax(1)[:, 1]

                # Visualize results
                vis.plot_ds_image(image_tensor, rois, class_scores)

NameError: name 'os' is not defined